In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")


display(x_train.head())
display(y_train.head())

train = pd.merge(x_train, y_train,on ='carID')
target = x_test.copy()

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [24]:
# print(target.shape)
# print()
# print(target.info())
# print()
# print(target.describe())
# print()
# print(target.isnull().sum())
# print(train.shape)
# print()
# print(train.info())
# print()
# print(train.describe())
# print()
# print(train.isnull().sum())

x = train.drop(['carID','price'], axis=1)
y = train['price']
target_x = target.drop('carID',axis=1)

# obj_col = x.select_dtypes(include='object').columns
# for col in obj_col:
#   print(col)
#   print(x[col].unique())
#   print(target_x[col].unique())
#   print(x[col].nunique())
#   print(target_x[col].nunique())

le_col = ['brand','model','year']
ohe_col = ['transmission','fuelType']
mms_col = ['mileage',	'tax','mpg',	'engineSize']

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

df = pd.concat([x, target_x], axis=0)

mms = MinMaxScaler()
df[mms_col] = mms.fit_transform(df[mms_col])

le = LabelEncoder()
for col in le_col:
  df[col] = le.fit_transform(df[col])

df = pd.get_dummies(df, columns=ohe_col,dtype=int)

x_pre = df.iloc[:len(x)]
target_x_pre = df.iloc[len(x):]

from sklearn.model_selection import train_test_split, GridSearchCV
rs=810
x_train, x_val, y_train, y_val = train_test_split(x_pre, y, test_size=0.2, random_state=rs)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=rs)

param = {'n_estimators':range(7,15),
         'max_depth':range(7,15)}
gscv = GridSearchCV(estimator = rf, param_grid= param, cv=5)
gscv.fit(x_train, y_train)
print(gscv.best_params_)
model = gscv.best_estimator_

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
pred_val = model.predict(x_val)
print(mean_squared_error(y_val, pred_val, squared=False))
print(mean_absolute_error(y_val, pred_val))
print(mean_absolute_error(y_val, pred_val))
print(r2_score(y_val, pred_val))

pred = model.predict(target_x_pre)
result = pd.DataFrame({'ID':target['carID'],
                       'pred':pred})
result.to_csv('result.csv',index=False)
print(result)

{'max_depth': 13, 'n_estimators': 14}
3960.3060960097296
2198.7243766503934
2198.7243766503934
0.9408684991806856
         ID          pred
0     12000  39440.306471
1     12001  24132.537906
2     12004  53765.100352
3     12013  15547.772646
4     12017  50837.904762
...     ...           ...
2667  19618  45742.261905
2668  19620  17465.214286
2669  19626  20690.109269
2670  19630  21977.630396
2671  19631   3291.070887

[2672 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
